In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 60
pd.options.display.max_rows = 500
import warnings
warnings.filterwarnings('ignore')

import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import os

import numpy as np

In [15]:
frac = 0.02

In [16]:
# Adulto equivalente. Cuanto cuesta la manutencion de las personas segun sexo y edad.
ad_eq = pd.read_csv('./../data/info/adulto_eq.csv')

#Importar canasta basica regional deflac
CB_ipc = pd.read_csv('./../data/info/CB_Reg_defl.csv')

# Load radio ref. Merge regiones.
# Anything that is AGLOMERADO 33 should be region Gran Buenos Aires

radio_ref = pd.read_csv('./../data/info/radio_ref.csv').merge(pd.read_csv('./../data/info/prov_regs.csv'), how = 'left')

radio_ref = pd.read_csv('./../data/info/radio_ref.csv')#.merge(aglo_labels)
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
radio_ref = radio_ref.merge(dpto_region)

# DPTO_Region = radio_ref[['DPTO', 'Region']].drop_duplicates()

## 1.  Resultados estaticos (se toma 1 año)

In [80]:
import glob
# path ='./data/RFReg_' # use your path
path ='./../data/yr_samples/RFReg_' # use your path

allFiles = []
for year in [str(s) for s in range(2019, 2020)]:
    allFiles += glob.glob(path +str(frac)+ '*'+str(year)+'*.csv')
    # Estos son los archivos que se usan para tener una figura estatica, corte donde no importa evol. temporal.

allFiles       


['./../data/yr_samples\\RFReg_0.02ARG2019-03-31.csv',
 './../data/yr_samples\\RFReg_0.02ARG2019-06-30.csv',
 './../data/yr_samples\\RFReg_0.02ARG2019-09-30.csv',
 './../data/yr_samples\\RFReg_0.02ARG2019-12-31.csv']

In [81]:
years = np.unique([int(f[-14:-10]) for f in  allFiles])

if len(years) == 1:
    yr_label = str(years[0])
else:
    yr_label = '-'.join([str(years.min()), str(years.max())])
    
yr_label

'2019'

In [61]:

df_parts = []
for quarter_Xy_file in sorted(allFiles):
    df_Q = pd.read_csv(quarter_Xy_file, 
                           usecols = ['PERSONA_REF_ID', 'HOGAR_REF_ID','RADIO_REF_ID', 'CONDACT', 'CAT_INAC', 'CAT_OCUP',
                                      'IX_TOT', 'H16', 'H15','P47T', 'P03','P02', 'P09','P10', 'DPTO'])
    df_Q['ANO4'] = int(quarter_Xy_file[-14:-10])
    df_Q['Q'] = quarter_Xy_file[-14:-4]
    df_parts += [df_Q]


In [63]:
# use means:
df = pd.concat(df_parts)

columnas_pesos = ['P47T']
df[columnas_pesos] = np.power(10, df[columnas_pesos]) - 1

ddf = dd.from_pandas(df, npartitions=50)

with ProgressBar():
    df = ddf.groupby(['PERSONA_REF_ID','Q']).mean().compute()

# Editar columnas
df['P10'] = 2 - df['P10']
df['P09'] = df.P09.replace(5, 4) #Polimodal tomado como secundario 

df = df.astype(int)
df['P0910'] = df.P09.astype(str) + df.P10.astype(str)
df['Grupo Etario'] = pd.cut(df.P03, np.arange(-1, 80, 3))#.round(-1)

df = df.reset_index()


[########################################] | 100% Completed | 22.4s


In [89]:
df_cb = df.merge(ad_eq).merge(DPTO_Region).merge(CB_ipc)

df_cb_hogares = df_cb.groupby(['HOGAR_REF_ID', 'Q'])[['P47T','CBA', 'CBT']].sum()
df_cb_hogares['Pobreza'] = df_cb_hogares['P47T'] < df_cb_hogares['CBT']
df_cb_hogares['Indigencia'] = df_cb_hogares['P47T'] < df_cb_hogares['CBA']
pobreza_hogares = df_cb_hogares[['P47T','CBA','CBT','Pobreza', 'Indigencia']].reset_index()
pobreza_hogares['gap_pobreza'] = pobreza_hogares.P47T - pobreza_hogares.CBT
pobreza_hogares['gap_indigencia'] = pobreza_hogares.P47T - pobreza_hogares.CBA
pobreza_hogares = pobreza_hogares.rename(columns = {'P47T': 'P47T_hogar'})

# df = df.sample(25000)
data = df.merge(pobreza_hogares, on = ['HOGAR_REF_ID', 'Q'])#, how = 'left')
data = data.rename(columns = {'P47T': 'P47T_persona'})

# data = data
data = data.merge(radio_ref[['RADIO_REF_ID', 'IDFRAC', 'PROV', 'NOMPROV', 'AGLOMERADO']].drop_duplicates())

if not os.path.exists('./../data/Pobreza/'):
    os.makedirs('./../data/Pobreza/')
data.to_csv('./../data/Pobreza/pobreza_'+'_'.join([str(frac), yr_label])+'.csv', index = False)

## 2. Resultados series de tiempo (se computa para todos y cada trimestre)

In [102]:
import glob
# path ='./data/RFReg_' # use your path
path ='./../data/yr_samples/RFReg_' # use your path

allFiles = []
for year in [str(s) for s in range(2003, 2021)]:
    allFiles += glob.glob(path +str(frac)+ '*'+str(year)+'*.csv')
    # Estos son los archivos que se usan para tener una figura estatica, corte donde no importa evol. temporal.


In [103]:
for quarter_Xy_file in sorted(allFiles):
    df = pd.read_csv(quarter_Xy_file, 
                           usecols = ['PERSONA_REF_ID', 'HOGAR_REF_ID','RADIO_REF_ID', 'CONDACT', 'CAT_INAC', 'CAT_OCUP',
                                      'IX_TOT', 'H16', 'H15','P47T', 'P03','P02', 'P09','P10', 'DPTO'])
    df['ANO4'] = int(quarter_Xy_file[-14:-10])
    q = quarter_Xy_file[-14:-4]
    df['Q'] = q
    print(q)
    
    columnas_pesos = ['P47T']
    df[columnas_pesos] = np.power(10, df[columnas_pesos]) - 1

#     ddf = dd.from_pandas(df, npartitions=50)
#     with ProgressBar():
#         df = ddf.groupby(['PERSONA_REF_ID']).mean().compute()

    # Editar columnas
    df['P10'] = 2 - df['P10']
    df['P09'] = df.P09.replace(5, 4) #Polimodal tomado como secundario 

#     df = df.astype(int)
    df['P0910'] = df.P09.astype(str) + df.P10.astype(str)
    df['Grupo Etario'] = pd.cut(df.P03, np.arange(-1, 80, 3))#.round(-1)

    df = df.reset_index()
    
    df_cb = df.merge(ad_eq).merge(DPTO_Region).merge(CB_ipc)

    df_cb_hogares = df_cb.groupby(['HOGAR_REF_ID', 'Q'])[['P47T','CBA', 'CBT']].sum()
    df_cb_hogares['Pobreza'] = df_cb_hogares['P47T'] < df_cb_hogares['CBT']
    df_cb_hogares['Indigencia'] = df_cb_hogares['P47T'] < df_cb_hogares['CBA']
    pobreza_hogares = df_cb_hogares[['P47T','CBA','CBT','Pobreza', 'Indigencia']].reset_index()
    pobreza_hogares['gap_pobreza'] = pobreza_hogares.P47T - pobreza_hogares.CBT
    pobreza_hogares['gap_indigencia'] = pobreza_hogares.P47T - pobreza_hogares.CBA
    pobreza_hogares = pobreza_hogares.rename(columns = {'P47T': 'P47T_hogar'})

    # df = df.sample(25000)
    data = df.merge(pobreza_hogares, on = ['HOGAR_REF_ID', 'Q'])#, how = 'left')
    data = data.rename(columns = {'P47T': 'P47T_persona'})

    # data = data
    data = data.merge(radio_ref[['RADIO_REF_ID', 'IDFRAC', 'PROV', 'NOMPROV', 'AGLOMERADO', 'Region']].drop_duplicates())
           
    if not os.path.exists('./../data/Pobreza/'):
        os.makedirs('./../data/Pobreza/')
    data.to_csv('./../data/Pobreza/pobreza_'+'_'.join([str(frac), 'q'+q])+'.csv', index = False)

2003-09-30
2003-12-31
2004-03-31
2004-06-30
2004-09-30
2004-12-31
2005-03-31
2005-06-30
2005-09-30
2005-12-31
2006-03-31
2006-06-30
2006-09-30
2006-12-31
2007-03-31
2007-06-30
2007-12-31
2008-03-31
2008-06-30
2008-09-30
2008-12-31
2009-03-31
2009-06-30
2009-09-30
2009-12-31
2010-03-31
2010-06-30
2010-09-30
2010-12-31
2011-03-31
2011-06-30
2011-09-30
2011-12-31
2012-03-31
2012-06-30
2012-09-30
2012-12-31
2013-03-31
2013-06-30
2013-09-30
2013-12-31
2014-03-31
2014-06-30
2014-09-30
2014-12-31
2015-03-31
2015-06-30
2016-06-30
2016-09-30
2016-12-31
2017-03-31
2017-06-30
2017-09-30
2017-12-31
2018-03-31
2018-06-30
2018-09-30
2018-12-31
2019-03-31
2019-06-30
2019-09-30
2019-12-31
2020-03-31
2020-06-30


In [ ]:
xx

In [ ]:

# df = pd.read_csv('./../data/info/ARG_'+str(frac)+'_deflac.csv')

# indice_precios = pd.read_csv('./../data/info/indice_pricestats_YQ.csv', index_col=0)
# indice_precios.index = pd.DatetimeIndex(indice_precios.index)
# indice_precios.index.name = 'Q'

# # El indice nos da los valores en pesos para cualquier trimestre. El ultimo es 3er trimestre 2018
# inx = indice_precios.tail(1).values[0][0]
# df['P47T'] = inx*df['P47T']

In [ ]:

# PERSONA = pd.read_csv('./../extracted_/PERSONA.csv', sep = ';', usecols = ['PERSONA_REF_ID', 'HOGAR_REF_ID']) # csv is too big, so it is dask-loaded. Not sure it's efficient thou
# PERSONA = PERSONA.loc[PERSONA.PERSONA_REF_ID.isin(df.PERSONA_REF_ID.values)]
# df = df.merge(PERSONA, how = 'left')

In [ ]:
# file_ = './data/sample_censo_table_f'+str(frac)+'ARG.csv'

# X_censo = pd.read_csv(file_, usecols = ['PERSONA_REF_ID', 'HOGAR_REF_ID','P03', 'CONDACT', 'AGLOMERADO']).fillna(0)
# df = df.drop(['P03', 'CONDACT', 'AGLOMERADO'], axis = 1).merge(X_censo)

# Distribucion del ingreso

In [ ]:
# import numpy as np
# fig, ax = plt.subplots(1, figsize = (5, 3))
# plt.hist(np.log10(df.loc[df.P47T > 100].P47T).values, 150, normed = 'pdf')#.groupby()
# plt.yscale('log')
# plt.show()

In [ ]:
# # fig, ax = plt.subplots(1, figsize = (10, 5))
# fig, ax = plt.subplots(1, figsize = (5, 3))
# df_ = df.groupby('HOGAR_REF_ID').sum()#.loc[df.P47T > 100]
# agg = df_.groupby(pd.cut(np.log10(df_.P47T), np.arange(2, 6, .05)))['P47T'].agg(['count','sum'])
# (agg/agg.sum()).plot(ax = ax)
# # ax.set_yscale('log')

# Calcular CBA y CBT

In [ ]:
# # Adulto equivalente. Cuanto cuesta la manutencion de las personas segun sexo y edad.
# ad_eq = pd.read_csv('./../data/info/adulto_eq.csv')

# # Canasta basica 'Alimentaria' y 'Total'
# CB = pd.read_csv('./../data/info/canasta.csv')

# valor_CB_2018_3 = CB.groupby(['Ano4', 'TRIMESTRE']).mean().loc[2018, 3]

# ad_eq['CBA'] = ad_eq['CB_EQUIV']*valor_CB_2018_3.CBA
# ad_eq['CBT'] = ad_eq['CB_EQUIV']*valor_CB_2018_3.CBT

# ad_eq.head()

### Mergear en sexo y edad

In [ ]:
# df_cb = df.merge(ad_eq).sort_values(by = 'HOGAR_REF_ID')

### Asi, podemos sumar por hogar, y comparar sus costos de canastas con los ingresos estimados

In [ ]:
# df_cb_hogares = df_cb.groupby('HOGAR_REF_ID')[['P47T','CBA', 'CBT']].sum()
# df_cb_hogares['Pobreza'] = df_cb_hogares['P47T'] < df_cb_hogares['CBT']
# df_cb_hogares['Indigencia'] = df_cb_hogares['P47T'] < df_cb_hogares['CBA']

In [ ]:
# # df_cb_hogares.reset_index()['Indigencia'].value_counts()
# df_cb_hogares.reset_index()['Pobreza'].value_counts()

# # # Pobreza en 17% de los hogares
# # 21049/(2945 + 119185)
# # # Indig. en 2% de los hogares
# # 2945/(2945 + 119185)

# pobreza_hogares = df_cb_hogares[['P47T','CBA','CBT','Pobreza', 'Indigencia']].reset_index()

In [ ]:
# pobreza_hogares.mean()

In [ ]:
# el erorr en una muestra de 1% podriaser de:
# +-0.22% en pobreza
# 0.01 en indigencia

In [ ]:
# 100*df_cb_hogares.reset_index()['Pobreza'].sum()/len(df_cb_hogares)
# 100*df_cb_hogares.reset_index()['Indigencia'].sum()/len(df_cb_hogares)


In [ ]:
# # hog_pobres = pobreza_hogares.loc[pobreza_hogares.Pobreza]
# pobreza_hogares['gap'] = pobreza_hogares.P47T - pobreza_hogares.CBT
# # hog_pobres['gap'] = hog_pobres.P47T - hog_pobres.CBT
# # hog_pobres.sum()

### Cargar data

In [ ]:

df = pd.read_csv('./../data/info/ARG_'+str(frac)+'_deflac.csv',
                 usecols = ['PERSONA_REF_ID', 'HOGAR_REF_ID','RADIO_REF_ID', 'P47T', 'P03','P02', 'P09','P10', 'DPTO'])

# Editar columnas
df['Grupo Etario'] = pd.cut(df.P03, np.arange(0, 80, 2))#.round(-1)
df['P10'] = 2 - df['P10']
df['P09'] = df.P09.replace(5, 4) #Polimodal tomado como secundario 
df['P0910'] = df.P09.astype(str) + df.P10.astype(str)


In [ ]:
20*len(df)

## Info Necesaria

In [ ]:
#Importar canasta basica reigonal deflac
CB_ipc = pd.read_csv('./../data/info/CB_Reg_defl.csv')
CB_ipc = CB_ipc.loc[CB_ipc.Q == '2019-12-31']
# CB_ipc = pd.read_csv('./../data/info/CB_Reg_defl.csv').groupby('Region').mean().reset_index()
# Se toma nivel de ultimo trimestre. Aca no hay trimestres.

# Adulto equivalente. Cuanto cuesta la manutencion de las personas segun sexo y edad.
ad_eq = pd.read_csv('./../data/info/adulto_eq.csv')

# Load radio ref. Merge regiones.
# Anything that is AGLOMERADO 33 should be region Gran Buenos Aires

# radio_ref = pd.read_csv('./../data/info/radio_ref.csv').merge(pd.read_csv('./../data/info/prov_regs.csv'), how = 'left')
radio_ref = pd.read_csv('./../data/info/radio_ref.csv')#.merge(aglo_labels)
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
radio_ref = radio_ref.merge(dpto_region)

DPTO_Region = radio_ref[['DPTO', 'Region']].drop_duplicates()

In [ ]:
# CB_ipc.loc[0, ['CBA', 'CBT']] = .3*CB_ipc.loc[0, ['CBA', 'CBT']]

## Generar columnas de Pobreza, etc

In [ ]:
df_cb = df.merge(ad_eq).merge(DPTO_Region).merge(CB_ipc)

df_cb_hogares = df_cb.groupby('HOGAR_REF_ID')[['P47T','CBA', 'CBT']].sum()
df_cb_hogares['Pobreza'] = df_cb_hogares['P47T'] < df_cb_hogares['CBT']
df_cb_hogares['Indigencia'] = df_cb_hogares['P47T'] < df_cb_hogares['CBA']
pobreza_hogares = df_cb_hogares[['P47T','CBA','CBT','Pobreza', 'Indigencia']].reset_index()
pobreza_hogares['gap'] = pobreza_hogares.P47T - pobreza_hogares.CBT
pobreza_hogares = pobreza_hogares.rename(columns = {'P47T': 'P47T_hogar'})

# df = df.sample(25000)

data = df.merge(pobreza_hogares, on = 'HOGAR_REF_ID', how = 'left')   
data = data.rename(columns = {'P47T': 'P47T_persona'})

data = data.merge(radio_ref[['RADIO_REF_ID', 'IDFRAC', 'PROV', 'AGLOMERADO']].drop_duplicates())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(1, figsize = (13, 6))
plt.hist(pobreza_hogares['gap'], np.arange(-30000, 200000, 1000), normed = True)
ax.axvline(0, c = 'k')
plt.xlim(-30000, 200000)
# plt.yscale('log')
plt.ylabel('PDF')
plt.xlabel('Ingreso del hogar por sobre Canasta Basica (AR$ 2018 Q3)')

plt.show()

In [ ]:
# pobreza_hogares.P47T_hogar = pobreza_hogares.P47T_hogar.round()
# pobreza_hogares.sort_values(by = 'HOGAR_REF_ID')

In [ ]:
pobreza_hogares.shape

## Incorporadas las columnas de pobreza e indigencia

In [ ]:
# df.HOGAR_REF_ID
# pobreza_hogares.HOGAR_REF_ID.nunique()
# data.shape

In [ ]:
# # df tiene ingresos de personas
# df.CONDACT.value_counts()
# df.P47T.nunique()

# # pobreza_hogares tiene ingresos de hogares
# pobreza_hogares.HOGAR_REF_ID.nunique()
# pobreza_hogares.P47T.nunique()

In [ ]:
# df = df.rename(columns = {'P47T': 'P47T_persona'})
# pobreza_hogares = pobreza_hogares.rename(columns = {'P47T': 'P47T_hogar'})
# data = df.merge(pobreza_hogares, on = 'HOGAR_REF_ID')
# data['Grupo Etario'] = pd.cut(data.P03, np.arange(0, 80, 3))#.round(-1)

# data['P10'] = 2 - data['P10']
# data['P09'] = data.P09.replace(5, 4) #Polimodal tomado como secundario 
# data['P0910'] = data.P09.astype(str) + data.P10.astype(str)


In [ ]:
100*data['Pobreza'].sum()/len(data)

In [ ]:
100*data['Indigencia'].sum()/len(data)


In [ ]:
# radio_ref = pd.read_csv('./../data/info/radio_ref.csv')
data = data.merge(radio_ref[['RADIO_REF_ID', 'IDFRAC', 'PROV', 'AGLOMERADO']].drop_duplicates())

In [ ]:
data

In [ ]:
import geopandas as gpd
dptos_gdf = gpd.read_file('./../../Documents/censo_geo/pxdptodatosok.shp')
dptos_gdf['personas'] = pd.to_numeric(dptos_gdf['personas'])

data['link'] = data['DPTO'].astype(str).str.zfill(5)

data['Pobreza'] = data['Pobreza'].astype(int)
pob_dpto = data.groupby('link')[['Pobreza']].mean().reset_index()

pob_dpto_gdf = gpd.GeoDataFrame(pob_dpto.merge(dptos_gdf))
# data.groupby('DPTO').count().iloc[:, 0].sort_values(ascending = False).head(10)

pob_dpto_gdf.loc[pob_dpto_gdf.personas < 3000, 'Pobreza'] = np.nan
pob_dpto_gdf = pob_dpto_gdf.fillna(pob_dpto_gdf.Pobreza.median())

In [ ]:
gdf = pob_dpto_gdf#.plot(column = 'Pobreza')

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

fig, ax = plt.subplots(1, figsize = (6, 14))
gdf.crs = {'init': 'epsg:4326'}
gdf.to_crs({'init': 'epsg:3395'}).fillna(0).plot(column = 'Pobreza', ax = ax, vmin = -.1, vmax = .66, cmap = 'RdYlGn_r')
plt.axis('off')
plt.show()

fig, ax = plt.subplots(1, figsize = (6, 14))
gdf.fillna(0).to_crs({'init': 'epsg:3395'}).plot(column = 'Pobreza', ax = ax, vmin = -.1, vmax = .66, cmap = 'RdYlGn_r', 
                                                edgecolor = '.4', lw = .4)
plt.axis('off')
plt.xlim(-7200000, -6250000)
plt.ylim(-5050000, -3850000)
plt.show()


fig, ax = plt.subplots(1, figsize = (6, 14))
gdf.fillna(0).to_crs({'init': 'epsg:3395'}).plot(column = 'Pobreza', ax = ax, vmin = 0, vmax = .66, cmap = 'RdYlGn_r',
                                                 edgecolor = '.4', lw = .4)
plt.axis('off')
plt.xlim(-6570000, -6450000)
plt.ylim(-4150000, -4020000)
plt.show()

## Usar Fracciones censales (bueno para algunos aglomerados)

In [ ]:
radio_ref.sample(20)

In [ ]:
# pob_frac = data.loc[data.AGLOMERADO.isin([32, 33, 2])].groupby(['IDFRAC','DPTO'])[['Pobreza']].mean().reset_index()
# pob_frac = data.loc[data.AGLOMERADO.isin([32])].groupby(['IDFRAC','DPTO'])[['Pobreza']].mean().reset_index()
pob_frac = data.loc[data.PROV.isin([82])].groupby(['IDFRAC','DPTO'])[['Pobreza']].mean().reset_index()
# pob_frac = data.loc[data.AGLOMERADO.isin([25])].groupby(['IDFRAC','DPTO'])[['Pobreza']].mean().reset_index()

frac_gdf = gpd.read_file('./../../Documents/censo_geo/pxfracdatosok.shp', )
pob_gdf = gpd.GeoDataFrame(pob_frac.merge(frac_gdf), crs = {'init': 'epsg:4326'})
# frac_gdf['personas'] = pd.to_numeric(frac_gdf['personas'])

# Fill na with median. Esp for areas of little pop.
# # data.groupby('DPTO').count().iloc[:, 0].sort_values(ascending = False).head(10)
# pob_gdf.loc[pob_gdf.personas < 3000, 'Pobreza'] = np.nan
# pob_gdf = pob_gdf.fillna(pob_dpto_gdf.Pobreza.median())

gdf = pob_gdf#.plot(column = 'Pobreza')

fig, ax = plt.subplots(1, figsize = (15, 14))
dptos_gdf.loc[dptos_gdf.codpcia.isin(['02', '06'])].to_crs({'init': 'epsg:3395'}).translate(yoff = 220, xoff = 30).plot(color = 'None', edgecolor = '.4', lw = .4, ax = ax, zorder = 10)
gdf.fillna(0).to_crs({'init': 'epsg:3395'}).plot(column = 'Pobreza', ax = ax, vmin = -.2, vmax = .5, cmap = 'RdYlGn_r')
plt.axis('off')
x1, y1, x2, y2 = tuple(gdf.fillna(0).to_crs({'init': 'epsg:3395'}).total_bounds)
plt.xlim(x1, x2)
plt.ylim(y1, y2)
plt.show()



### Subir mapa en nivel frac censal.

In [ ]:
pob_frac = data.groupby(['IDFRAC','DPTO'])[['Pobreza']].mean().reset_index()

# frac_gdf = gpd.read_file('./../../Documents/censo_geo/pxfracdatosok.shp', )
pob_gdf = gpd.GeoDataFrame(pob_frac.merge(frac_gdf), crs = {'init': 'epsg:4326'})


# frac_gdf['personas'] = pd.to_numeric(frac_gdf['personas'])

# Fill na with median. Esp for areas of little pop.
# # data.groupby('DPTO').count().iloc[:, 0].sort_values(ascending = False).head(10)
# pob_gdf.loc[pob_gdf.personas < 3000, 'Pobreza'] = np.nan
# pob_gdf = pob_gdf.fillna(pob_dpto_gdf.Pobreza.median())


In [ ]:
# # niveles de actividad por dpto
# data.groupby(['DPTO'])['CONDACT'].value_counts().groupby('DPTO').apply(lambda x: x/x.sum())
# data.groupby(['DPTO'])['P0910'].value_counts().groupby('DPTO').apply(lambda x: x/x.sum())

# data.nunique()

# data.P0910.value_counts()

aglo_data = data.loc[data.AGLOMERADO.isin([32, 33])]

income = aglo_data.groupby(['DPTO', 'IDFRAC'])['gap'].agg(['count', 'mean', 'median']).reset_index()
income_gdf = gpd.GeoDataFrame(income.merge(frac_gdf), 
                           crs = {'init': 'epsg:4326'})



gdf = income_gdf#.plot(column = 'Pobreza')

fig, ax = plt.subplots(1, figsize = (15, 14))
dptos_gdf.loc[dptos_gdf.codpcia.isin(['02', '06'])].to_crs({'init': 'epsg:3395'}).translate(yoff = 220).plot(color = 'None', edgecolor = '.4', lw = .6, ax = ax, zorder = 10)
gdf.fillna(0).to_crs({'init': 'epsg:3395'}).plot(column = 'median', ax = ax, vmin = 0, vmax = 35000, cmap = 'RdYlGn')

plt.axis('off')
x1, y1, x2, y2 = tuple(gdf.fillna(0).to_crs({'init': 'epsg:3395'}).total_bounds)
plt.xlim(x1, x2)
plt.ylim(y1, y2)
plt.show()

In [ ]:
cat_cols = ['V01', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10',
       'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'PROP',
       'P02', 'P05', 'P07', 'P08', 'P09', 'P10', 'CAT_OCUP',
       'CAT_INAC', 'CH07', 'PP07G1', 'PP07G2', 'PP07G3', 'PP07G4', 'PP07G_59',
       'PP07H', 'PP07I', 'PP07J', 'PP07K',
       'AGLOMERADO', 'CONDACT', 'Grupo Etario'] # IX_TOT
# value_cols = 'P47T' #?

vn = pd.read_csv('./../data/info/VARIABLE_NAMES.csv')
# vn['Pregunta'] = vn['Pregunta'].str.strip()#.str.replace("'", "") Don't run

In [ ]:
data.nunique()

In [ ]:
condicion = 'Pobreza'
for cat in cat_cols[-1:]:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 5))
    info = data.groupby([cat, condicion])[['PERSONA_REF_ID', 'HOGAR_REF_ID']].nunique()
    info_pct = (100*(info.unstack()/info.unstack().sum()).stack())
    sns.barplot(x=cat, y="PERSONA_REF_ID", hue=condicion, data=info_pct.reset_index(), ax = ax1)
    xlab = ''
    try:
        xlab = vn.loc[vn.Censo == cat]['Pregunta'].values[0]
    except:
        try:
            xlab = vn.loc[vn.EPH == cat]['Pregunta'].values[0]
        except:
            pass
    ax1.set(xlabel= xlab, ylabel = 'porcentaje de PERSONAS')
    ax1.tick_params(axis='x', rotation=45)
    sns.barplot(x=cat, y="PERSONA_REF_ID", hue=condicion, data=info.reset_index(), ax = ax2)
    ax2.set(xlabel= xlab, ylabel = 'porcentaje de HOGARES \n con alguien')
    ax2.tick_params(axis='x', rotation=45)
    plt.show()